# Spotify Data Analysis Project
## 1. Overview about the project 
##### "The Spotify Million Playlist Dataset Challenge consists of a dataset and evaluation to enable research in music recommendations. It is a continuation of the RecSys Challenge 2018, which ran from January to July 2018. 
##### **Goal:** Develop a system which takes a playlist, and predict the next songs on the playlist.
##### **Dataset:** https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge#dataset

## 2. Load The Dataset: Using Apache Sparks to Load All 1000 Files.
**Note**: Below are the steps to convert json data into a suitale structured format to analyze in Apache Sparks. 

In [174]:
!pip install pandas sqlalchemy pyspark 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 27.3 MB/s eta 0:00:00
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
  Running setup.py install for pyspark ... done


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SpotifyDataAnalysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/30 16:58:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#drop any corrupt record (it occurs while sparks parsing the dataset there are some unexpected issues)
df = spark.read.option("mode", "DROPMALFORMED").json('/home/student/ttngo/SpotifyDataProject/data/mpd.slice.*.json')

#### Lets Try With A Subset of Dataset fiest (24 files)

In [241]:
import os
import json
directory = "/home/student/ttngo/SpotifyDataProject/data/"
# Get a list of all JSON files in the directory
files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
files_subset = files[:24]

In [242]:
data = []

# Loop through each file in the subset
for file in files_subset:
    # Open each file
    with open(file, 'r') as f:
        # Load the data from the file and append it to the list
        data.append(json.load(f))

In [243]:
import pandas as pd
df= pd.DataFrame(data)

In [244]:
df.shape

(24, 2)

In [245]:
import pandas as pd
# Initialize an empty list to store the playlists
playlists = []
# Loop through each file's data
for file_data in data:
    # Append each playlist to the list
    playlists.extend(file_data['playlists'])

# Convert the list of playlists to a DataFrame
dataframe = pd.DataFrame(playlists)


In [407]:
%store dataframe

Stored 'dataframe' (DataFrame)


In [408]:
%store playlists

Stored 'playlists' (list)


## 3. Requesting More Information From Spotify API

In [44]:
client_id = 'c468fcc1bfb343a5972d0fae6cc23802'
client_secret ='2d996f020fe644f0b5e53fa257414c2e'

In [178]:
import requests
# Define the endpoint
url = 'https://accounts.spotify.com/api/token'
# Define the headers
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
}
# Define the data
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Make the POST request
response = requests.post(url, headers=headers, data=data)

# Get the access token from the response
access_token = response.json()['access_token']

# Print the access token
print('Access token:', access_token)


Access token: BQAJI0YF1A_GbuddWEK5f6-ZttAX7Pdg5lmz7z1zddnCGjXr1oCr4qkSGpxOHCksGHFOsazCTzwH-IGfRY9zXyi7whgL5EKx3QUMBValzHrJXuaxtPo


## 4. Create SQl Database 
**Note:** These database will store all the information about these table: Playlists, Artists, Tracks, Playlists_Tracks, Albums (including the additonal informations just requested).
##### **Steps to create Database:** Create columns (addiotnal information included), fetch additional informations, insert value

In [56]:
import sqlite3

### 5.1 Playlist Tables

In [378]:
conn = sqlite3.connect('my.db')
cur = conn.cursor()
#CREATE
create_playlist_table = '''
CREATE TABLE IF NOT EXISTS Playlists (
    pid INT PRIMARY KEY,
    name TEXT,
    collaborative TEXT,
    modified_at INT,
    num_tracks INT,
    num_albums INT,
    num_followers INT
    )
    '''
cur.execute(create_playlist_table)
conn.commit()
print("Table 'Playlists' created successfully.")

Table 'Playlists' created successfully.


**Checking**: The total rows in this Playlists table should be 24k because there are 24 json files and each files contains 1000 playlists.

In [249]:
#INSERT VALUES
# Prepare data for tracks insertion
playlists_data=[]
for playlist in playlists:
    playlists_data.append((
        playlist['name'],
        playlist['collaborative'],
        playlist['pid'],
        playlist['modified_at'],
        playlist['num_tracks'],
        playlist['num_albums'],
        playlist['num_followers']
    ))

In [286]:
len(playlists_data)

24000

In [379]:
#Inserting each keys in the tracks_data into the table
insert_playlists_query = '''
INSERT OR IGNORE INTO Playlists (name, collaborative, pid, modified_at, num_tracks, num_albums, num_followers)
VALUES (?, ?, ?, ?, ?, ?, ?)
'''

cur.executemany(insert_playlists_query, playlists_data)
conn.commit()
cur.close()
conn.close()

In [380]:
#CHECK THE SIZE OF TABLE (NUMBER OF ROWS)
conn = sqlite3.connect('my.db')
# Create a cursor object using the connection
cur = conn.cursor()

cur.execute('SELECT COUNT(*) FROM Playlists')
row_count = cur.fetchone()[0]

# Print the number of rows
print(f'Number of rows in the Playlists table: {row_count}')

# Close the cursor and connection
cur.close()
conn.close()

Number of rows in the Playlists table: 24000


In [383]:
conn = sqlite3.connect('my.db')
cur = conn.cursor()

# Execute a SELECT query to fetch one row from the Playlists table
cur.execute('SELECT * FROM Playlists LIMIT 1')
row = cur.fetchone()

# Print the fetched row
print("First row from the Playlists table:")
print(row)

# Close the cursor and connection
cur.close()
conn.close()


First row from the Playlists table:
(57000, 'reggaeton', 'false', 1509321600, 69, 54, 1)


### 5.2: Tracks Tables

**Notes:** The length of tracks_data is greater than the len of the Tracks table because tracks table handle duplicates value. In other words, there are some repeated song along multiple playlists will be removed.

In [171]:
# CREATE A LIST OF UNIQUE TRACKS
track_uris=[]
for playlist in playlists:
    for track in playlist['tracks']:
        track_uri= track['track_uri']
        if track_uri not in track_uris:
            track_uris.append(track_uri)
len(track_uris)

35279

#### Before create a track table, we need to fetch more information about track beside the track metadata(pos, track_uri, track_name,..). Track's audio feature would be helpful. The code below is fucntion to fetch the track's audio feature.

In [177]:
def fetch_audio_features(track_chunks, access_token, delay=0.2):
    audio_features_url = 'https://api.spotify.com/v1/audio-features'
    headers = {'Authorization': f'Bearer {access_token}'}
    audio_features = []

    for chunk in track_chunks:
        track_ids = ','.join(chunk)
        params = {'ids': track_ids}
        response = requests.get(audio_features_url, headers=headers, params=params)

        if response.status_code == 200:
            audio_features.extend(response.json()['audio_features'])
        elif response.status_code == 429:  # Rate limit exceeded
            retry_after = int(response.headers.get('Retry-After', delay))
            print(f"Rate limit exceeded, retrying after {retry_after} seconds...")
            time.sleep(retry_after)
            continue
        else:
            print(f"Failed to fetch audio features: {response.status_code} - {response.text}")

        # Introduce a small delay between requests to avoid hitting rate limits
        time.sleep(delay)

    return audio_features

In [180]:
track_ids = [uri.split(':')[-1] for uri in track_uris]
chunk_size = 50
track_chunks = [track_ids[i:i+chunk_size] for i in range(0, len(track_ids), chunk_size)]
# Fetch audio features in chunks
access_token = 'BQAJI0YF1A_GbuddWEK5f6-ZttAX7Pdg5lmz7z1zddnCGjXr1oCr4qkSGpxOHCksGHFOsazCTzwH-IGfRY9zXyi7whgL5EKx3QUMBValzHrJXuaxtPo'
audio_features = fetch_audio_features(track_chunks, access_token)

In [196]:
len(audio_features)

35279

#### Now, let's create tables which contains both track's metadata and track's audio features.
**Note:** we can map the correspond track_uri in 'Track metadata' to uri in 'Track's audio features'. In this way, we ca n the correspond audio feature to a unquie track. 

In [412]:
#CREATE TRACKS TABLE.
conn = sqlite3.connect('my.db')
cur = conn.cursor()

create_tracks_table = '''
CREATE TABLE IF NOT EXISTS Tracks (
    pos INTEGER,
    artist_name TEXT,
    track_uri TEXT PRIMARY KEY,
    artist_uri TEXT,
    track_name TEXT,
    album_uri TEXT,
    duration_ms INTEGER,
    album_name TEXT,
    danceability REAL,
    energy REAL,
    key INTEGER,
    loudness REAL,
    mode INTEGER,
    speechiness REAL,
    acousticness REAL,
    instrumentalness REAL,
    liveness REAL,
    valence REAL,
    tempo REAL,
    type TEXT,
    id TEXT,
    track_href TEXT,
    analysis_url TEXT,
    time_signature INTEGER
    )
    '''
cur.execute(create_tracks_table)
conn.commit()

In [368]:
tracks_data= []  

for playlist in playlists:
    for track in playlist['tracks']:
        tracks_data.append({
            'pos': track.get('pos'),            
            'artist_name': track.get('artist_name'),  
            'track_uri': track.get('track_uri'),      
            'artist_uri': track.get('artist_uri'),   
            'track_name': track.get('track_name'),    
            'album_uri': track.get('album_uri'),      
            'duration_ms': track.get('duration_ms'),  
            'album_name': track.get('album_name')     
        })

In [369]:
len(tracks_data)

1605538

In [397]:
combined_data = []

# Assuming both tracks_data and audio_features have the same length
for track_meta, audio_feature in zip(tracks_data, audio_features):
    track_data = {
        'pos': track_meta.get('pos'),
        'artist_name': track_meta.get('artist_name'),
        'track_uri': track_meta.get('track_uri'),
        'artist_uri': track_meta.get('artist_uri'),
        'track_name': track_meta.get('track_name'),
        'album_uri': track_meta.get('album_uri'),
        'duration_ms': track_meta.get('duration_ms'),
        'album_name': track_meta.get('album_name'),
        'danceability': audio_feature.get('danceability', 0) if audio_feature else 0,
        'energy': audio_feature.get('energy', 0) if audio_feature else 0,
        'key': audio_feature.get('key', 0) if audio_feature else 0,
        'loudness': audio_feature.get('loudness', 0) if audio_feature else 0,
        'mode': audio_feature.get('mode', 0) if audio_feature else 0,
        'speechiness': audio_feature.get('speechiness', 0) if audio_feature else 0,
        'acousticness': audio_feature.get('acousticness', 0) if audio_feature else 0,
        'instrumentalness': audio_feature.get('instrumentalness', 0) if audio_feature else 0,
        'liveness': audio_feature.get('liveness', 0) if audio_feature else 0,
        'valence': audio_feature.get('valence', 0) if audio_feature else 0,
        'tempo': audio_feature.get('tempo', 0) if audio_feature else 0,
        'type': audio_feature.get('type', '') if audio_feature else '',
        'id': audio_feature.get('id', '') if audio_feature else '',
        'track_href': audio_feature.get('track_href', '') if audio_feature else '',
        'analysis_url': audio_feature.get('analysis_url', '') if audio_feature else '',
        'time_signature': audio_feature.get('time_signature', 0) if audio_feature else 0
    }
    combined_data.append(track_data)

# After creating combined_data, proceed with insertion into the database


In [398]:
len(combined_data)

35279

In [411]:
%store combined_data

Stored 'combined_data' (list)


In [413]:
for track_data in combined_data:
    cur.execute("""
        INSERT OR REPLACE INTO Tracks (
            pos, artist_name, track_uri, artist_uri, track_name, album_uri,
            duration_ms, album_name, danceability, energy, key, loudness,
            mode, speechiness, acousticness, instrumentalness, liveness,
            valence, tempo, type, id, track_href, analysis_url, time_signature
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        track_data['pos'], track_data['artist_name'], track_data['track_uri'],
        track_data['artist_uri'], track_data['track_name'], track_data['album_uri'],
        track_data['duration_ms'], track_data['album_name'], track_data['danceability'],
        track_data['energy'], track_data['key'], track_data['loudness'],
        track_data['mode'], track_data['speechiness'], track_data['acousticness'],
        track_data['instrumentalness'], track_data['liveness'], track_data['valence'],
        track_data['tempo'], track_data['type'], track_data['id'],
        track_data['track_href'], track_data['analysis_url'], track_data['time_signature']
    ))

conn.commit()
cur.close()
conn.close()

In [414]:
#CHECK THE SIZE OF TABLE (NUMBER OF ROWS)
conn = sqlite3.connect('my.db')
# Create a cursor object using the connection
cur = conn.cursor()

cur.execute('SELECT COUNT(*) FROM Tracks')
row_count = cur.fetchone()[0]

# Print the number of rows
print(row_count)

# Close the cursor and connection
cur.close()
conn.close()

21517


### 5.3: Artists Tables

In [289]:
# Initialize an empty list to store artist URIs
artist_uris = []

In [ ]:
# Loop through each playlist in your playlists data
for playlist in playlists:
    for track in playlist['tracks']:
        artist_uri = track['artist_uri']
        if artist_uri not in artist_uris:
            artist_uris.append(artist_uri)

In [ ]:
print(len(artist_uris))

**Note:** since there are 10,262 artists in total (among 24,000 playlist), we will divide them into chunks of 50 ids and use Get Several Artist to proccess 50 artists a time. So there will be 10,262/50 = 206 chunks. The limit of request of API is 10-15 per seconds. 

In [148]:
import requests
import time
# url= 'https://api.spotify.com/v1/artists/{
url = 'https://api.spotify.com/v1/artists'
access_token = 'BQADgqBy9cwybs6zqF8brF2prmAKm2sOe3mdDvJcKyh0aLj_VKgsdBexmJsvtlg48mnPHqFVby0x4esAnKBfSs0NeiAnfLTuLKsUb9IQ9NFGr5AQVJo'

In [152]:
def fetch_artists_info(artist_chunks, access_token, delay=0.2):
    headers = {'Authorization': f'Bearer {access_token}'}
    artists_info = []

    for chunk in artist_chunks:
        artist_ids = ','.join(chunk)
        params = {'ids': artist_ids}  # Combine artist IDs into a comma-separated string
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            artists_info.extend(response.json()['artists'])
        elif response.status_code == 429:  # Rate limit exceeded
            retry_after = int(response.headers.get('Retry-After', delay))
            print(f"Rate limit exceeded, retrying after {retry_after} seconds...")
            time.sleep(retry_after)
            continue
        else:
            print(f"Failed to fetch artists info: {response.status_code} - {response.text}")

        # Introduce a small delay between requests to avoid hitting rate limits
        time.sleep(delay)  # Adjust delay as needed based on API guidelines

    return artists_info

In [ ]:
# Now divide into chunk of 50 ids
artist_ids = [uri.split(':')[-1] for uri in artist_uris]
chunk_size = 50
artist_chunks = [artist_ids[i:i+chunk_size] for i in range(0, len(artist_ids), chunk_size)]

# Fetch artists' information in chunks
artists_info = fetch_artists_info(artist_chunks, access_token)

In [168]:
print((artists_info[0]['id'], (artists_info[0]['name'],artists_info[0]['genres'],artists_info[0]['followers'])))

('3jOstUTkEu2JkjvRdBA5Gu', ('Weezer', ['alternative rock', 'modern power pop', 'modern rock', 'permanent wave', 'rock'], {'href': None, 'total': 3604348}))


In [302]:
len(artists_info)

10262

In [410]:
%store artists_info

Stored 'artists_info' (list)


In [304]:
artists_info_dataframe = pd.DataFrame(artists_info)
artists_info_dataframe[:2]

,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 3604348}","[alternative rock, modern power pop, modern ro...",https://api.spotify.com/v1/artists/3jOstUTkEu2...,3jOstUTkEu2JkjvRdBA5Gu,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Weezer,73,artist,spotify:artist:3jOstUTkEu2JkjvRdBA5Gu
1,{'spotify': 'https://open.spotify.com/artist/2...,"{'href': None, 'total': 3746249}","[modern alternative rock, modern rock, pov: in...",https://api.spotify.com/v1/artists/26T3LtbuGT1...,26T3LtbuGT1Fu9m0eRq5X3,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Cage The Elephant,74,artist,spotify:artist:26T3LtbuGT1Fu9m0eRq5X3


In [405]:
%store artists_info_dataframe

Stored 'artists_info_dataframe' (DataFrame)


**Note:** Useful columns: artist_uri, artist_name, popularity, genres, followers. 

In [306]:
#Create the table caled 'Artists'
conn = sqlite3.connect('my.db')
cur = conn.cursor()
create_artists_table = '''
CREATE TABLE IF NOT EXISTS Artists (
    artist_uri TEXT PRIMARY KEY,
    artist_name TEXT,
    followers INTEGER,
    genres TEXT,
    popularity INTEGER
)
    '''
cur.execute(create_artists_table)
conn.commit()

In [307]:
for artist in artists_info:
    artist_uri = artist['uri']
    artist_name = artist['name']
    followers = artist['followers']['total']
    genres = ', '.join(artist['genres']) if artist['genres'] else None  # Handle case where genres may be empty
    popularity = artist['popularity']

    # Insert data into table
    cur.execute('''INSERT OR IGNORE INTO Artists 
                   (artist_uri, artist_name, followers, genres, popularity) 
                   VALUES (?, ?, ?, ?, ?)''', (artist_uri, artist_name, followers, genres, popularity))

# Commit changes and close connection
conn.commit()
cur.close()
conn.close()

In [310]:
# TRY TO RETREIEV INFO: TOP 5 POPULAR ARTISTS

# Connect to SQLite database
conn = sqlite3.connect('my.db')
cur = conn.cursor()

# Query to select top 5 popular artists
query = '''
    SELECT artist_name, popularity
    FROM Artists
    ORDER BY popularity DESC
    LIMIT 5
'''

# Execute the query
cur.execute(query)

# Fetch all rows from the result set
top_artists = cur.fetchall()

# Print or process the results
for rank, (artist_name, popularity) in enumerate(top_artists, start=1):
    print(f"Rank {rank}: {artist_name} (Popularity: {popularity})")

# Close the cursor and connection
cur.close()
conn.close()


Rank 1: Taylor Swift (Popularity: 100)
Rank 2: Drake (Popularity: 94)
Rank 3: Billie Eilish (Popularity: 94)
Rank 4: Bad Bunny (Popularity: 94)
Rank 5: The Weeknd (Popularity: 93)


In [308]:
#CHECK THE SIZE OF TABLE (NUMBER OF ROWS)
conn = sqlite3.connect('my.db')
# Create a cursor object using the connection
cur = conn.cursor()

cur.execute('SELECT COUNT(*) FROM Artists')
row_count = cur.fetchone()[0]

# Print the number of rows
print(row_count)

# Close the cursor and connection
cur.close()
conn.close()

10238


### 5.3: Playlists_Tracks Tables

In [415]:
#CREATE
conn = sqlite3.connect('my.db')
cur = conn.cursor()

create_playtrack_table = '''
CREATE TABLE IF NOT EXISTS Playlists_Tracks (
    pid INT,
    track_uri TEXT,
    pos INT,
    FOREIGN KEY(pid) REFERENCES Playlists(pid),
    FOREIGN KEY(track_uri) REFERENCES Tracks(track_uri),
    PRIMARY KEY(pid, track_uri)
)
'''
cur.execute(create_playtrack_table)

In [416]:
#INSERT VALUES INTO PLAYLISTS_TRACKS
playlist_track_data=[]
for playlist in playlists:
    for track in playlist["tracks"]:
        playlist_track_data.append((
            playlist['pid'],
            track['track_uri'],
            track['pos']
        ))
        
insert_track = '''
INSERT OR IGNORE INTO Playlists_Tracks (pid, track_uri, pos)
VALUES (?, ?, ?)
'''

cur.executemany(insert_track, playlist_track_data)
conn.commit()
cur.close()
conn.close()

In [417]:
#CHECK THE SIZE OF TABLE (NUMBER OF ROWS)
conn = sqlite3.connect('my.db')
# Create a cursor object using the connection
cur = conn.cursor()

cur.execute('SELECT COUNT(*) FROM Playlists_Tracks')
row_count = cur.fetchone()[0]

# Print the number of rows
print(row_count)

# Close the cursor and connection
cur.close()
conn.close()

1584468


### 5.4: Artists_Tracks Table

In [419]:
conn = sqlite3.connect('my.db')
cur= conn.cursor()

create_artists_tracks_table = '''
CREATE TABLE IF NOT EXISTS TrackArtists (
    track_uri TEXT,
    artist_uri TEXT,
    PRIMARY KEY (track_uri, artist_uri),
    FOREIGN KEY (track_uri) REFERENCES Tracks(track_uri),
    FOREIGN KEY (artist_uri) REFERENCES Artists(artist_uri)
)
'''
cur.execute(create_artists_tracks_table )

In [421]:
#INSERT VALUES INTO PLAYLISTS_TRACKS
track_artists_data=[]
for data in combined_data:
    track_artists_data.append((
        data['artist_uri'],
        data['track_uri']
    ))
    
        
        
insert_track_artist = '''
INSERT OR IGNORE INTO TrackArtists (track_uri, artist_uri)
VALUES (?, ?)
'''

cur.executemany(insert_track_artist, track_artists_data)
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

In [422]:
#CHECK THE SIZE OF TABLE (NUMBER OF ROWS)
conn = sqlite3.connect('my.db')
# Create a cursor object using the connection
cur = conn.cursor()

cur.execute('SELECT COUNT(*) FROM TrackArtists')
row_count = cur.fetchone()[0]

# Print the number of rows
print(row_count)

# Close the cursor and connection
cur.close()
conn.close()

21517


## 5. CREATE RECOMMENDATION SYSTEM.
**Approach 1:** KNN Algorithm. In this approach, we will take a given point (track's audio feature), and then predict the 5 nearest points to it (Euclidean Distance). 

### 1. Loading The Data In Tracks Table .

In [428]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('my.db')
query = "SELECT * FROM Tracks"
tracks_df = pd.read_sql_query(query, conn)

conn.close()


### 2. Preprocessing The Data
* Extracts the audio features (like danceability, energy, etc.) from the track data.
* Normalizes these features to ensure they are on a similar scale, which is necessary for calculating similarity.

In [431]:
# Selecting only prefer audio feature 
selected_af= tracks_df[[ 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo']]

In [432]:
# Normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
selected_af_scaled = scaler.fit_transform(selected_af)

In [433]:
audio_features_scaled_df = pd.DataFrame(selected_af_scaled, columns=selected_af.columns)

In [435]:
audio_features_scaled_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.642713,0.435,0.272727,0.811479,1.0,0.032124,0.003293,0.000007,0.071357,0.463636,0.321308
1,0.643725,0.676,0.181818,0.907469,1.0,0.027254,0.081024,0.000000,0.083518,0.275758,0.481257
2,0.491903,0.903,0.090909,0.904739,1.0,0.038549,0.000283,0.000056,0.131658,0.307071,0.557529
3,0.589069,0.677,0.636364,0.821834,1.0,0.178238,0.031325,0.000007,0.122613,0.207071,0.818357
4,0.764170,0.866,1.000000,0.896927,0.0,0.090155,0.440763,0.001330,0.258291,0.567677,0.460209


### 3. Select 10 playlists and Calcuate The Centroids of Each PLaylist
* 1. It’s going through each playlist ID in a list of selected playlist IDs.
* 2. For each playlist ID, select correspond playlist
* 3. for each playlist, select track_uri 
* 4. Find the correspond track features in tracks_df(danceability, liveness, ,...) in a playlist
* 5. Calcuate the mean of all track's audio feature ( mean of danceability, mean of liveness,...)
* 6. Store this centroid in a dictionary, using the playlist ID as the key

In [454]:
selected_playlists= playlists[:10]

In [475]:
first_10_playlist_ids = [playlist['pid'] for playlist in selected_playlists[:10]]
first_10_playlist_ids

[57000, 57001, 57002, 57003, 57004, 57005, 57006, 57007, 57008, 57009]

In [469]:
playlist_centroids = {}

for playlist_id in first_10_playlist_ids:
    # Find the playlist in the playlists list
    playlist = next((p for p in playlists if p['pid'] == playlist_id), None)
    
    if playlist:
        # Extract track URIs from the playlist
        track_uris = [track['track_uri'] for track in playlist['tracks']]
        
        # Filter tracks_df based on track URIs
        playlist_tracks = audio_features_scaled_df[tracks_df['track_uri'].isin(track_uris)]
        
        # Calculate centroid (average feature vector) for the playlist
        playlist_centroid = playlist_tracks.mean(axis=0)
        
        # Store centroid in dictionary
        playlist_centroids[playlist_id] = playlist_centroid


In [474]:
for pid, centroid in playlist_centroids.items():
    print(f"Playlist ID: {pid}")
    print(centroid)
    print()

Playlist ID: 57000
danceability        0.603606
energy              0.663870
key                 0.504611
loudness            0.813922
mode                0.681159
speechiness         0.077738
acousticness        0.203282
instrumentalness    0.054247
liveness            0.191071
valence             0.524297
tempo               0.515215
dtype: float64

Playlist ID: 57001
danceability        0.538682
energy              0.669243
key                 0.446640
loudness            0.785865
mode                0.869565
speechiness         0.063289
acousticness        0.169835
instrumentalness    0.139514
liveness            0.148835
valence             0.539521
tempo               0.539656
dtype: float64

Playlist ID: 57002
danceability        0.583300
energy              0.678009
key                 0.477686
loudness            0.790773
mode                0.727273
speechiness         0.071395
acousticness        0.227097
instrumentalness    0.070545
liveness            0.202267
valence     

In [487]:
centroids_df = pd.DataFrame.from_dict(playlist_centroids, orient='index', columns=selected_af.columns)
centroids_df


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
57000,0.603606,0.663870,0.504611,0.813922,0.681159,0.077738,0.203282,0.054247,0.191071,0.524297,0.515215
57001,0.538682,0.669243,0.446640,0.785865,0.869565,0.063289,0.169835,0.139514,0.148835,0.539521,0.539656
57002,0.583300,0.678009,0.477686,0.790773,0.727273,0.071395,0.227097,0.070545,0.202267,0.566889,0.530627
57003,0.625000,0.643593,0.324675,0.807141,0.857143,0.090533,0.411013,0.118911,0.198248,0.477778,0.472411
57004,0.596449,0.615488,0.477273,0.768330,0.666667,0.072215,0.285133,0.114837,0.194778,0.485690,0.529804
57005,0.490457,0.510786,0.415584,0.799945,0.785714,0.036247,0.419550,0.054533,0.131141,0.439199,0.554371
57006,0.491228,0.394250,0.507576,0.735019,0.666667,0.035130,0.667335,0.463867,0.132990,0.294949,0.508919
57007,0.545058,0.521950,0.465517,0.763914,0.793103,0.047956,0.396706,0.133664,0.192767,0.460794,0.510241
57008,0.535407,0.620486,0.524351,0.808181,0.696429,0.070946,0.269459,0.067221,0.178964,0.477166,0.499038
57009,0.547012,0.581623,0.443357,0.774052,0.661538,0.061041,0.343021,0.201234,0.197968,0.444584,0.518342


In [488]:
from sklearn.neighbors import NearestNeighbors
knn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')  
knn_model.fit(audio_features_scaled_df)               

ImportError: cannot import name 'sort_graph_by_row_values' from 'sklearn.neighbors._base' (/home/student/ttngo/.local/lib/python3.11/site-packages/sklearn/neighbors/_base.py)

In [ ]:
recommended_tracks = {}

for playlist_id, centroid in centroids_df.iterrows():
    # Reshape centroid to match expected input for kneighbors
    centroid_array = np.array(centroid).reshape(1, -1)
    
    # Find nearest neighbors
    distances, indices = knn_model.kneighbors(centroid_array)

    recommended_tracks[playlist_id] = indices.flatten()[1:]  # Exclude the first index
    
# Output recommended tracks (indices)
for playlist_id, tracks_indices in recommended_tracks.items():
    print(f"Playlist ID: {playlist_id}")
    print("Recommended Track Indices:", tracks_indices)
    print("Recommended Tracks:")
    print(tracks_df.iloc[tracks_indices][['track_uri', 'track_name']])
    print()